In [1]:
from dotenv import load_dotenv
import os
from langchain.chat_models import AzureChatOpenAI
import warnings
warnings.filterwarnings("ignore")

In [2]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model = AzureChatOpenAI(temperature=0, deployment_name="chat")

# Cadenas de utilidad
* Son funciones que ya tienen un proposito muy especifico; generar un resumen a partir de textos, resolver preguntas, crear una conversación con memoria o sin memoria.


In [3]:
model.invoke("Que tipo de clima hay en Chile?")

AIMessage(content='Chile tiene una gran variedad de climas debido a su extensa longitud y a la presencia de la cordillera de los Andes y el océano Pacífico. En general, se pueden identificar cinco zonas climáticas principales:\n\n1. Desierto: En el norte de Chile se encuentra el desierto de Atacama, considerado el más árido del mundo. Aquí el clima es extremadamente seco y las temperaturas son altas durante el día y frías durante la noche.\n\n2. Mediterráneo: En la zona central de Chile, que incluye ciudades como Santiago, Valparaíso y Viña del Mar, el clima es mediterráneo, con veranos cálidos y secos e inviernos suaves y húmedos.\n\n3. Templado lluvioso: En el sur de Chile, desde la región de La Araucanía hasta la región de Los Lagos, el clima es templado lluvioso. Aquí las temperaturas son más frescas durante todo el año y las precipitaciones son abundantes, especialmente en invierno.\n\n4. Frío: En la región de Aysén y Magallanes, en el extremo sur de Chile, el clima es frío, con t

In [4]:
# load_summarize_chain
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
# Cargamos el pdf
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()


chain = load_summarize_chain(model, chain_type="map_reduce", verbose=False)  #stuff o map_reduce

chain.run(docs)

'The article examines the development of autonomous agents using large language models. It covers the various aspects of these agents, such as planning, memory, and tool use, and includes case studies and proof-of-concept demos. The article acknowledges the challenges and limitations of using language models and provides additional resources for further reading.'

In [5]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY in spanish:"""
prompt = PromptTemplate.from_template(prompt_template)
llm_chain = LLMChain(llm=model, prompt=prompt)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")
print(stuff_chain.run(docs))

El artículo discute la idea de construir agentes autónomos utilizando modelos de lenguaje grandes (LLM) como controladores principales. Se presentan varios ejemplos de demostraciones de concepto, como AutoGPT y GPT-Engineer, que demuestran el potencial de los LLM para resolver problemas complejos. El sistema de agente autónomo consta de tres componentes principales: planificación, memoria y uso de herramientas. La planificación implica descomponer tareas complejas en subobjetivos más pequeños y mejorar los resultados a través de la autorreflexión. La memoria incluye la memoria a corto plazo y la memoria a largo plazo, que permite al agente retener y recuperar información. El uso de herramientas implica que el agente aprenda a utilizar API externas para obtener información adicional. Sin embargo, existen desafíos en cuanto a la longitud limitada del contexto, la planificación a largo plazo y la confiabilidad de la interfaz de lenguaje natural.


In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:/Users/Ignacia/Desktop/Ignacia/WORKBENCH/BootCamp_Langchain_Actividades/Scripts/Chains/resources/GCP.pdf")
pages = loader.load_and_split()

In [7]:
len(pages)

275

In [8]:
print(stuff_chain.run(pages[:10]))

El libro "Visualizing" de Priyanka Vergadia es una guía visual de los conceptos clave de Google Cloud. El libro cubre desde los conceptos básicos de la infraestructura de la nube hasta temas más avanzados como análisis de datos, ciencia de datos, aprendizaje automático e inteligencia artificial. Está dirigido a entusiastas de Google Cloud, arquitectos de soluciones en la nube, tomadores de decisiones de TI y profesionales de ingeniería de datos y aprendizaje automático. El libro utiliza ilustraciones para hacer que los conceptos técnicos sean más accesibles y fáciles de entender.


In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        length_function = len,
        chunk_overlap = 250
)

documents = text_splitter.split_documents(data)

In [10]:
len(documents)

1767

In [11]:
print(stuff_chain.run(documents[:10]))

El resumen conciso del texto proporcionado es que se trata de un libro titulado "Visualizing" escrito por Priyanka Vergadia, que ofrece referencias ilustradas para ingenieros y arquitectos de la nube. El libro está protegido por derechos de autor y publicado por Google, LLC y John Wiley & Sons, Inc. El texto también menciona que no se puede reproducir ninguna parte de la publicación sin permiso previo del editor. Además, se advierte que el contenido del libro no garantiza la exactitud o integridad y no se debe considerar como asesoramiento profesional.


In [12]:
from langchain_community.document_loaders import TextLoader
# from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('C:/Users/Ignacia/Desktop/Ignacia/WORKBENCH/BootCamp_Langchain_Actividades/Scripts/Chains/resources/cuento.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, embedding_function)
chain_qa = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever = db.as_retriever(search_kwargs={"k": 2})
)
chain_qa.run("Entregame un resumen del cuento")



'El cuento narra la historia de una niña llamada Sara, quien vive en un pequeño pueblo rodeado de un bosque misterioso. Un día, Sara descubre una vieja puerta de madera oculta en el bosque y decide abrirla. Al cruzarla, se encuentra en un mundo mágico lleno de maravillas.\n\nEn su aventura, Sara se enfrenta a desafíos emocionantes, como rescatar a un unicornio atrapado en una red de arañas gigantes y enfrentarse a un laberinto de espejos enigmático. Finalmente, llega al castillo de un malvado mago que ha estado causando problemas en el reino. Con ingenio y determinación, Sara derrota al mago y libera a las criaturas que había capturado.\n\nGracias a su valentía y bondad, Sara se gana el corazón de todos en el mundo mágico. Cuando decide regresar a su pueblo, las criaturas la acompañan hasta la puerta que la llevará de regreso a casa y prometen visitarla en sueños y aventuras.'

In [13]:
# import
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma

# load the document and split it into chunks
loader = TextLoader("C:/Users/Ignacia/Desktop/Ignacia/WORKBENCH/BootCamp_Langchain_Actividades/Scripts/Chains/resources/cuento.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# query it
query = "de que se trata el cuento?"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

A medida que avanzaba, Sara se enfrentaba a desafÃ­os y obstÃ¡culos emocionantes. RescatÃ³ a un unicornio atrapado en una red de araÃ±as gigantes, y luego se enfrentÃ³ a un enigmÃ¡tico laberinto de espejos donde tenÃ­a que tomar decisiones difÃ­ciles para encontrar la salida.

Finalmente, llegÃ³ al castillo de un malvado mago que habÃ­a estado causando problemas en el reino. Con ingenio y determinaciÃ³n, Sara derrotÃ³ al mago y liberÃ³ a las criaturas que habÃ­a capturado.

Con su valentÃ­a y bondad, Sara se ganÃ³ el corazÃ³n de todos en el mundo mÃ¡gico. Cuando decidiÃ³ regresar a su pueblo, las criaturas la acompaÃ±aron hasta la puerta que la llevarÃ­a de regreso a casa. Prometieron visitarla en sueÃ±os y aventuras.


# Cadenas fundacionales

* Proporcionan un marco para llevar a cabo una serie de transformaciones de manera ordenada y estructurada.

In [14]:
from langchain.chains import LLMMathChain
llm_math = LLMMathChain.from_llm(model)
llm_math.run("Cuanto es 2*2**45?")

'Answer: 70368744177664'

In [15]:
import re

def limpiar_texto(entradas: dict) -> dict:
    texto = entradas["texto"]

    # Eliminamos los emojis utilizando un amplio rango unicode
    # Ten en cuenta que esto podría potencialmente eliminar algunos caracteres válidos que no son en inglés
    patron_emoji = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticonos
        "\U0001F300-\U0001F5FF"  # símbolos y pictogramas
        "\U0001F680-\U0001F6FF"  # símbolos de transporte y mapas
        "\U0001F1E0-\U0001F1FF"  # banderas (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE,
    )
    texto = patron_emoji.sub(r'', texto)

    # Removemos las URLs
    patron_url = re.compile(r'https?://\S+|www\.\S+')
    texto = patron_url.sub(r'', texto)

    return {"texto_limpio": texto}

In [16]:
from langchain.chains import TransformChain

chain_clean= TransformChain(
    input_variables=["texto"],
    output_variables=["texto_limpio"],
    transform=limpiar_texto
)

In [17]:
chain_clean.run('Visiten está página https://facebook.com/home 🙈')

'Visiten está página  '

In [18]:
plantilla = """Parafrasea este texto:

{texto_limpio}

En el estilo de una persona informal de {estilo}.

Parafraseado: """

prompt = PromptTemplate(
    input_variables=["texto_limpio", "estilo"],
    template=plantilla
)

In [19]:
from langchain.chains import LLMChain

chain_style = LLMChain(
    llm=model,
    prompt=prompt,
    output_key='texto_final'
)

In [20]:
# from torch.utils.data.dataset import ChainDataset
from langchain.chains import SequentialChain

cadena_sequential = SequentialChain(
    chains=[chain_clean, chain_style],
    input_variables=["texto", "estilo"],
    output_variables=["texto_final"]
)

In [21]:
texto_entrada = """
¡Monterrey es una ciudad impresionante! 🏙️
Es conocida por su impresionante paisaje de montañas ⛰️ y su vibrante cultura norteña.
¡No olvides visitar el famoso Museo de Arte Contemporáneo (MARCO)!
🖼️ Si eres fanático del fútbol, no puedes perderte un partido de los Rayados o de los Tigres. ⚽
Aquí te dejo algunos enlaces para que puedas conocer más sobre esta maravillosa ciudad:
https://visitamonterrey.com, https://museomarco.org, https://rayados.com, https://www.tigres.com.mx.
¡Monterrey te espera con los brazos abiertos! 😃🇲🇽
"""

In [22]:
cadena_sequential({'texto': texto_entrada, 'estilo': 'Argentina'})

{'texto': '\n¡Monterrey es una ciudad impresionante! 🏙️\nEs conocida por su impresionante paisaje de montañas ⛰️ y su vibrante cultura norteña.\n¡No olvides visitar el famoso Museo de Arte Contemporáneo (MARCO)!\n🖼️ Si eres fanático del fútbol, no puedes perderte un partido de los Rayados o de los Tigres. ⚽\nAquí te dejo algunos enlaces para que puedas conocer más sobre esta maravillosa ciudad:\nhttps://visitamonterrey.com, https://museomarco.org, https://rayados.com, https://www.tigres.com.mx.\n¡Monterrey te espera con los brazos abiertos! 😃🇲🇽\n',
 'estilo': 'Argentina',
 'texto_final': '¡Monterrey es una ciudad increíble, che! \nEs re conocida por su paisaje de montañas impresionante y su cultura norteña re vibrante.\n¡No te olvides de visitar el re famoso Museo de Arte Contemporáneo (MARCO)!\nSi sos fanático del fútbol, no te podés perder un partido de los Rayados o de los Tigres. \nAcá te dejo unos links para que puedas conocer más sobre esta ciudad re copada:\n   \n¡Monterrey te e